In [1]:
import numpy as np
# just performs simple automatic differentiation

In [2]:
class CompNode:
    def __init__(self, tape):
        # make sure that the gradient tape knows us
        tape.add(self)
        self.output = 0
    
    # perform the intended operation 
    # and store the result in self.output
    def forward(self):
        pass
    
    # assume that self.gradient has all the information 
    # from outgoing nodes prior to calling backward
    # -> perform the local gradient step with respect to inputs
    def backward(self):
        pass
    
    # needed to be initialized to 0 
    def set_gradient(self, gradient):
        self.gradient = gradient
        
    # receive gradients from downstream nodes     
    def add_gradient(self, gradient):
        self.gradient += gradient
    
class ConstantNode(CompNode):
    def __init__(self, value, tape):
        self.value = value
        super().__init__(tape)
        
    def forward(self):
        self.output = self.value
    
    def backward(self):
        # nothing to do here
        pass
    
class Multiply(CompNode):
    
    def __init__(self, left : CompNode, right : CompNode, tape):
        self.left = left
        self.right = right
        super().__init__(tape)
        
    def forward(self):
        self.output = self.left.output * self.right.output
        
    # has to know how to locally differentiate multiplication
    def backward(self):
        self.left.add_gradient(self.right.output * self.gradient)
        self.right.add_gradient(self.left.output * self.gradient)
        
class Tape:
    
    def __init__(self):
        self.computations = []
        
    def add(self, compNode : CompNode):
        self.computations.append(compNode)
        
    def forward(self):
        for compNode in self.computations:
            compNode.forward()
            
    def backward(self):
        # first initialize all gradients to zero 
        for compNode in self.computations:
            compNode.set_gradient(0)
            
        # we need to invert the order    
        self.computations.reverse()    
        # last node gets a default value of one for the gradient
        self.computations[0].set_gradient(1)
        for compNode in self.computations:
            compNode.backward()

In [3]:
t = Tape()
a = ConstantNode(2,t)
b = ConstantNode(3,t)

o = Multiply(a, b, t)
f = Multiply(ConstantNode(5, t), o, t)
t.forward()

In [4]:
print(f.output)

30


In [5]:
# start reverse mode autodifferentiation
t.backward()

In [6]:
# now inspect the gradients 
print(f.gradient)
print(o.gradient)
print(a.gradient)
print(b.gradient)

1
5
15
10


### A diamond-shaped graph that makes use of the multivariate chain rule

In [7]:
t = Tape()
x = ConstantNode(3,t)
y = ConstantNode(2,t)
z = ConstantNode(1,t)

h1 = Multiply(x, y, t)
h2 = Multiply(y, z, t)

h = Multiply(h1, h2, t)
o = Multiply(h, h, t)
t.forward()

In [8]:
t.backward()
print(h.gradient)
print("--")
print(h1.gradient)
print(h2.gradient)
print("--")
print(x.gradient)
print(y.gradient)
print(z.gradient)

24
--
48
144
--
96
288
288


now with an explicit operation for taking the square.

In [9]:
class Square(CompNode):
    
    def __init__(self, x : CompNode, tape : Tape):
        self.x = x
        super().__init__(tape)
        
    def forward(self):
        self.output = self.x.output**2
        
    # has to know how to locally differentiate x^2
    def backward(self):
        self.x.add_gradient( (2*self.x.output) * self.gradient)


In [10]:
t = Tape()
x = ConstantNode(3,t)
y = ConstantNode(2,t)
z = ConstantNode(1,t)

h1 = Multiply(x, y, t)
h2 = Multiply(y, z, t)

h = Multiply(h1, h2, t)
o = Square(h, t)
t.forward()

In [11]:
t.backward()
print(h.gradient)
print("--")
print(h1.gradient)
print(h2.gradient)
print("--")
print(x.gradient)
print(y.gradient)
print(z.gradient)

24
--
48
144
--
96
288
288


## Application to a small neural network graph
To really connect the dots, we need to instantiate our small framework to the problem of differentiating the loss function of a neural network. Again, this will just be a toy example with two features that are multiplied by weights, fed through a sigmoid activation and compared to a target output.

<img src="graphfornetwork.png" width = "80%"> 

In [12]:
# first, we need new functions for inverting a node's output, the sigmoid, and an Add operation
class Invert(CompNode):
    
    def __init__(self, x : CompNode, tape : Tape):
        self.x = x
        super().__init__(tape)
        
    def forward(self):
        self.output = (-1)*self.x.output
        
    # has to know how to locally differentiate x * (-1)
    def backward(self):
        self.x.add_gradient( (-1) * self.gradient)
        
class Add(CompNode):
    
    def __init__(self, left : CompNode, right : CompNode, tape):
        self.left = left
        self.right = right
        super().__init__(tape)
        
    def forward(self):
        self.output = self.left.output + self.right.output
        
    # has to know how to locally differentiate addition (SPOILER: it just distributes its incoming gradient)
    # d (l + r) / d l = 1 
    # d (l + r) / d r = 1 
    def backward(self):
        self.left.add_gradient(self.gradient)
        self.right.add_gradient(self.gradient)
        
class Sigmoid(CompNode):
    
    def __init__(self, x : CompNode, tape : Tape):
        self.x = x
        super().__init__(tape)
        
    def forward(self):
        self.output = 1. / (1. + np.exp(-self.x.output))
        
    # has to know how to locally differentiate sigmoid (which is easy, given the output)
    # d sigmoid(x) / d x = sigmoid(x)*(1-sigmoid(x)) 
    def backward(self):
        local_gradient = self.output * (1. - self.output)
        self.x.add_gradient( local_gradient * self.gradient)

Now we're ready to implement the graph for some input features and weights.
<img src="graphfornetwork_start.png" width = "70%"> 

In [13]:
gt = Tape()
x1 = ConstantNode(2.,gt)
w1 = ConstantNode(0.4,gt)
x2 = ConstantNode(3.,gt)
w2 = ConstantNode(-0.2,gt)
t = ConstantNode(1.,gt)

h1 = Multiply(x1, w1, gt)
h2 = Multiply(x2, w2, gt)

h = Add(h1, h2, gt)
y = Sigmoid(h,gt)

t_inv = Invert(t, gt)
e = Add(y, t_inv, gt)
l = Square(e, gt)
gt.forward()

In [14]:
print("---")
print(h1.output)
print(h2.output)
print(h.output)
print(y.output)
print("---")
print(e.output)
print(l.output)

---
0.8
-0.6000000000000001
0.19999999999999996
0.5498339973124778
---
-0.45016600268752216
0.2026494299756622


In [15]:
gt.backward()
print(l.gradient)
print("--")
print(e.gradient)
print(t_inv.gradient)
print(y.gradient)
print("--")
print(h.gradient)
print("--")
print(h1.gradient)
print(h2.gradient)
print("--")
print(w1.gradient)
print(w2.gradient)
print("--")
print(x1.gradient)
print(x2.gradient)
print("--")
print(t.gradient)

1
--
-0.9003320053750443
-0.9003320053750443
-0.9003320053750443
--
-0.22284709227322685
--
-0.22284709227322685
-0.22284709227322685
--
-0.4456941845464537
-0.6685412768196806
--
-0.08913883690929075
0.044569418454645376
--
0.9003320053750443


In [16]:
# a test implementation of a CompNode for ReLU
class ReLU(CompNode):
    
    def __init__(self, x : CompNode, tape : Tape):
        self.x = x
        super().__init__(tape)
        
    def forward(self):
        self.output = np.max( self.x.output, 0)
        
    # has to know how to locally differentiate sigmoid 
	# (which is easy, given the output)
    # d sigmoid(x) / d x = sigmoid(x)*(1-sigmoid(x)) 
    def backward(self):
        local_gradient = self.output * (1. - self.output)
        self.x.add_gradient( local_gradient * self.gradient)